# Level 1 - MODELS 

# ML Models Trained using Scikit-Learn

## Changes made:

1. Included Random state to create reproducible models (models which give the same result everytime we run )

2. Calculated F1 score on set-1 also along with set-2 & set-3

TRAIN : set-1 (50 %) 

VAL :  
1. set-2 (25 %)
2. set-3 (25% )
3. Test data




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
set1 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set1.csv')
set2 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set2.csv')
set3 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set3.csv')

test = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/tweets_test_features.csv')

In [ ]:
print(set1.shape)
print(set2.shape)
print(set3.shape)
print(test.shape)

(3806, 25)
(1903, 25)
(1904, 25)
(3263, 24)


In [ ]:
set1[:2]

,keyword,location,text,target,keyword_missing,location_missing,word_count,unique_word_count,mean_word_length,char_count,punctuation_count,alphabets,digits,all_nouns,special_nouns,normal_nouns,verbs,all_nouns_len,special_nouns_len,normal_nouns_len,verbs_len,simple_clean,no_stopword,lemmatized,no_pronoun
0,destroy,missing,@SarniamakChris @Hromadske @kasiadear33 how s...,1,0,1,22,20,5.318182,139,3,112,2,@SarniamakChris @Hromadske @kasiadear33 countr...,@SarniamakChris @Hromadske @kasiadear33,countries world say world security,can destroy,8,3,5,2,sarniamakchris hromadske kasiadear how silly ...,sarniamakchris hromadske kasiadear silly count...,sarniamakchris hromadske kasiadear silly count...,sarniamakchris hromadske kasiadear silly count...
1,thunder,missing,L B #Oklahoma #Thunder DURANT NBA ADIDAS OKLAH...,0,0,1,19,19,6.263158,137,10,105,4,L B # Oklahoma # Thunder DURANT NBA ADIDAS OKL...,L B # Oklahoma # Thunder NBA ADIDAS SWINGMAN J...,DURANT OKLAHOMA CITY THUNDER # http://t.co/T81...,NaN,19,13,6,0,l b oklahoma thunder durant nba adidas oklahom...,l b oklahoma thunder durant nba adidas oklahom...,l b oklahoma thunder durant nba adida oklahoma...,l b oklahoma thunder durant nba adida oklahoma...


In [ ]:
x1 = set1.drop('target',axis=1)
x2 = set2.drop('target',axis=1)
x3 = set3.drop('target',axis=1)

In [ ]:
y1 = set1.target
y2 = set2.target
y3 = set3.target

In [ ]:
!pip install -q category_encoders

     |████████████████████████████████| 81kB 3.3MB/s 


In [ ]:
import category_encoders as ce

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

In [ ]:
def training_model(m):

  model = Pipeline([('enc', ce.LeaveOneOutEncoder()),
                    ('clf', m ) ])
  model.fit(x1,y1) # Fit once

  x2_proba = model.predict_proba(x2)
  x3_proba = model.predict_proba(x3)
  test_proba = model.predict_proba(test)

  pred_x1 = model.predict(x1)
  print('F1 SCORE for set-2 : %.4f'%f1_score(y1, pred_x1, average='binary'))

  pred_x2 = model.predict(x2)
  print('F1 SCORE for set-2 : %.4f'%f1_score(y2, pred_x2, average='binary'))
  print('Predictions = ',pred_x2[:5])

  pred_x3 = model.predict(x3)
  print('F1 SCORE for set-3 : %.4f'%f1_score(y3, pred_x3, average='binary'))
  
  return x2_proba, x3_proba, test_proba

# MODELS

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(learning_rate=0.05,subsample = 0.01,min_samples_split = 5, min_samples_leaf = 5,max_depth= 1, random_state=1)
gb_x2, gb_x3, gb_test = training_model(gb)

F1 SCORE for set-2 : 0.6856
F1 SCORE for set-2 : 0.6640
Predictions =  [0 1 0 1 0]
F1 SCORE for set-3 : 0.6858


In [ ]:
gb_x2[:5] # 0,1

array([[0.53250648, 0.46749352],
       [0.23117721, 0.76882279],
       [0.61181238, 0.38818762],
       [0.36062681, 0.63937319],
       [0.78679872, 0.21320128]])

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear', max_iter=500, random_state=1)
lr_x2, lr_x3, lr_test = training_model(lr)

F1 SCORE for set-2 : 0.7511
F1 SCORE for set-2 : 0.6889
Predictions =  [1 1 0 1 0]
F1 SCORE for set-3 : 0.7146


In [ ]:
lr_x2 # 1,0   ### Only Logistic Regression returns proba_array for 1 first  & then for 0.

array([[0.41808077, 0.58191923],
       [0.16483688, 0.83516312],
       [0.60736177, 0.39263823],
       ...,
       [0.42587888, 0.57412112],
       [0.46469778, 0.53530222],
       [0.41476654, 0.58523346]])

In [ ]:
from sklearn import svm
svc = svm.SVC(kernel = 'linear',probability=True, random_state=1)
svc_x2, svc_x3, svc_test = training_model(svc)

F1 SCORE for set-2 : 0.7469
F1 SCORE for set-2 : 0.6929
Predictions =  [1 1 0 1 0]
F1 SCORE for set-3 : 0.7185


In [ ]:
svc_x2[:5] #0,1

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
xtra_tree = ExtraTreesClassifier(random_state=1)  
tree_x2, tree_x3, tree_test = training_model(xtra_tree)

F1 SCORE for set-2 : 0.9739
F1 SCORE for set-2 : 0.6603
Predictions =  [1 1 0 1 0]
F1 SCORE for set-3 : 0.6775


In [ ]:
tree_x2[:5] # 0,1

In [ ]:
from sklearn.naive_bayes import BernoulliNB
bayes = BernoulliNB(binarize=0.4)
bayes_x2, bayes_x3, bayes_test =training_model(bayes)

F1 SCORE for set-2 : 0.7078
F1 SCORE for set-2 : 0.6546
Predictions =  [1 1 1 1 0]
F1 SCORE for set-3 : 0.6620


In [ ]:
bayes_x2[:5] # 0 1

In [ ]:
!pip install -q catboost

     |████████████████████████████████| 64.8MB 46kB/s 


In [ ]:
from catboost import CatBoostClassifier
catboost = CatBoostClassifier(iterations=3,
                           learning_rate= 0.05,
                           depth=1)
cat_x2, cat_x3, cat_test = training_model(catboost)

0:	learn: 0.6819854	total: 49.2ms	remaining: 98.4ms
1:	learn: 0.6728154	total: 50.8ms	remaining: 25.4ms
2:	learn: 0.6639790	total: 52.3ms	remaining: 0us
F1 SCORE for set-2 : 0.7075
F1 SCORE for set-2 : 0.6829
Predictions =  [0 1 0 1 0]
F1 SCORE for set-3 : 0.7044


In [ ]:
cat_x2[:5] # 0 1

In [ ]:
abc

# Probability df

In [ ]:
def proba_list(pred_proba_arr):
  prob_list = pred_proba_arr.tolist()
  disaster_probability = []
  not_a_disaster_probability = []
  for i in prob_list:
    disaster_probability.append(i[0])
    not_a_disaster_probability.append(i[1])
  return disaster_probability, not_a_disaster_probability

# Create 3 dataframes of probas

In [ ]:
gb0, gb1 = proba_list(gb_x2)
lr1, lr0 = proba_list(lr_x2)
svc0, svc1 = proba_list(svc_x2)
bayes0, bayes1 = proba_list(bayes_x2)
tree0, tree1 = proba_list(tree_x2)
cat0, cat1 = proba_list(cat_x2)

In [ ]:
x2_df = pd.DataFrame({'gb1' : gb1, 'gb0' : gb0,
                      'lr1' : lr1, 'lr0' : lr0,
                      'svc1' : svc1,'svc0' : svc0,
                      'bayes1' : bayes1, 'bayes0' : bayes0,
                      'tree1' : tree1, 'tree0' : tree0,
                      'cat1' : cat1, 'cat0' : cat0
                      })

In [ ]:
gb0, gb1 = proba_list(gb_x3)
lr1, lr0 = proba_list(lr_x3)
svc0, svc1 = proba_list(svc_x3)
bayes0, bayes1 = proba_list(bayes_x3)
tree0, tree1 = proba_list(tree_x3)
cat0, cat1 = proba_list(cat_x3)

In [ ]:
x3_df = pd.DataFrame({'gb1' : gb1, 'gb0' : gb0,
                      'lr1' : lr1, 'lr0' : lr0,
                      'svc1' : svc1,'svc0' : svc0,
                      'bayes1' : bayes1, 'bayes0' : bayes0,
                      'tree1' : tree1, 'tree0' : tree0,
                      'cat1' : cat1, 'cat0' : cat0
                      })

In [ ]:
gb0, gb1 = proba_list(gb_test)
lr1, lr0 = proba_list(lr_test)
svc0, svc1 = proba_list(svc_test)
bayes0, bayes1 = proba_list(bayes_test)
tree0, tree1 = proba_list(tree_test)
cat0, cat1 = proba_list(cat_test)

In [ ]:
test_df = pd.DataFrame({'gb1' : gb1, 'gb0' : gb0,
                      'lr1' : lr1, 'lr0' : lr0,
                      'svc1' : svc1,'svc0' : svc0,
                      'bayes1' : bayes1, 'bayes0' : bayes0,
                      'tree1' : tree1, 'tree0' : tree0,
                      'cat1' : cat1, 'cat0' : cat0
                      })

In [ ]:
x2_df.shape, x3_df.shape, test_df.shape

# Save a copy of ml_probas for future ensemble model

In [ ]:
x2_df.to_csv('set2_probas.csv', index=False)
!cp set2_probas.csv "/content/drive/My Drive/Tweets/5. Predict Probas"

x3_df.to_csv('set3_probas.csv', index=False)
!cp set3_probas.csv "/content/drive/My Drive/Tweets/5. Predict Probas"

test_df.to_csv('test_probas.csv', index=False)
!cp test_probas.csv "/content/drive/My Drive/Tweets/5. Predict Probas"

In [ ]:
x2_df[:2]

In [ ]:
x3_df[:2]

In [ ]:
test_df[:2]